<a href="https://colab.research.google.com/github/ailton-felix/Combinacoes/blob/master/colab/clusterization/Clusteriza%C3%A7%C3%A3o_din%C3%A2mica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Firebase e Pytools

In [ ]:
from google.colab import auth
import sys
import subprocess

# Autenticação é necessária para clonar
auth.authenticate_user()

!gcloud source repos clone github_hand-talk_pytools --project='ht-community' && pip install -r /content/github_hand-talk_pytools/requirements.txt
sys.path.insert(0, '/content/github_hand-talk_pytools')

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=t08oQR4YcvOjcN7M6YNBSXsZsTiBWM&prompt=consent&access_type=offline&code_challenge=AOywi79wVxzdFUK7aBSS-5jHaST6zoiz_fZ9DvHkWME&code_challenge_method=S256

Enter verification code: 4/1AX4XfWg5TzJsvFYh-ufXhNwXb9lVcryDezYQMxv6bqpi1_lVHnZuJNEdmJY


In [ ]:
#@title Definição de variáveis globais  { form-width: "30%" }

WORKSPACE = "HT-BZS" #@param ["HT-Annoy", "HT-BZS", "development", "HT-BZS-JOAB", "HT-ASL"] {allow-input: true}


In [4]:
from handtalk.community import Community

# A linha abaixo é necessária somente fora do colab
# handtalk.config.key_firebase = path/para/key.json
cmt = Community(WORKSPACE)

{'Faça o upload do arquivo json do service accounts do community. Se não tiver um, peça ao Thadeu.'}


Saving community-service-account.json to community-service-account.json


# Clusterização automática

Para o módulo HTube, isso é realizado automáticamente (todas as decisões são persistidas).

Para o caso do módulo match segments, temos, caso dois segmentos sejam votados como semelhantes:

- Nenhum dos segmentos pertence a um cluster;

- Um dos segmentos pertence a um cluster;

- Os dois segmentos pertencem a cluster:

PROBLEMA: TF-IDF

### Backlog de sinais e segmentos com cluster antes de clusterizar

In [5]:
# Carregar TF-IDF
cmt.storage.download(f'modelos/automatic_cluster/TF_IFD_{WORKSPACE}.pkl','./')

# Load the dictionary back from the pickle file.
import pickle

tf_idf = pickle.load( open( f"TF_IFD_{WORKSPACE}.pkl", "rb" ) )

In [6]:
from datetime import datetime
from copy import deepcopy
from datetime import datetime,timedelta


#template de sinais criados
sign_template = {'createdAt' : datetime.now(),
    'createdBy' : cmt.get_document_reference('users','6ukYAMNKaHYC2xmxXLGEmx8L2mF2'),
    'currentAnimation': None,
    'currentAnimationHasErrors': None,
    'description': '',
    'glosa': '',
    'hasCluster': True,
    'isAnimated': False,
    'isAnimating': {'lastUpdate': None, 'user': None, 'value': False},
    'isCropped': False,
    'isCropping': {'lastUpdate': None, 'taskName': None, 'value': False},
    'isDefinitive': False,
    'isExtractedKeypoints': False,
    'isExtractingKeypoints': {'lastUpdate': None,
    'taskName': None,
    'value': False},
    'isFingerspell': False,
    'isStudioFileRecorded': False,
    'isStudioFileRecording': {'lastUpdate': None, 'user': None, 'value': False},
    'numberOfSegments': 0,
    'oralLanguageId': 'eng' if WORKSPACE != 'HT-BZS' else 'por',
    'origin': {'endFrame': 0, 'startFrame': 0, 'video': None},
    'popularity': 0,
    'primarySegment': None,
    'promoted': False,
    'relatedWords': [],
    'searchReference': '',
    'searchTerms': [],
    'signLanguageId': 'ase' if WORKSPACE != 'HT-BZS' else 'bzs',
    'stars': 0,
    'userPrimarySegment': False,
    'words': [] }

# funçãão de ordenaçãão dos termos do tf-idf
def ordered_words(sentence_tf_idf):
    return sorted([(w, round(score, 3)) for w, score in sentence_tf_idf], key=lambda x: x[1], reverse=True)

# stop words do tf-idf
stop = ['em', 'do', 'da', 'de', 'pro', 'pra', 'no', 'dos', 'na', 'os', 'à', 'pela', 'nas', 'num',\
    'das', 'numa', 'nos', 'às', 'pelas', 'as', 'uns', 'umas', 'lhe', 'é', 'o', 'a', 'tom', 'mary', 'toms', 'marys',
    'is', 'are', 'am', 'at', 'an', 'of', 'the'] if WORKSPACE == 'HT-BZS' \
    else \
    ['em', 'da', 'de', 'pro', 'pra', 'no', 'dos', 'na', 'os', 'à', 'pela', 'nas', 'num',\
    'das', 'numa', 'nos', 'às', 'pelas', 'as', 'uns', 'umas', 'lhe', 'é', 'o', 'a', 'tom', 'mary', 'toms', 'marys',
    'is', 'are', 'am', 'at', 'an', 'of', 'the']

# Coleta dos votos recentes

In [7]:
votes = cmt.list_longer_collection('votes')

🍒 Sys[out] » Total votes 30000 query time 15.13s 
🍒 Sys[out] » Total votes 60000 query time 26.58s 
🍒 Sys[out] » Total votes 90000 query time 38.16s 
🍒 Sys[out] » Total votes 120000 query time 49.6s 
🍒 Sys[out] » Total votes 150000 query time 61.19s 
🍒 Sys[out] » Total votes 180000 query time 72.29s 
🍒 Sys[out] » Total votes 210000 query time 83.97s 
🍒 Sys[out] » Total votes 240000 query time 95.38s 
🍒 Sys[out] » Total votes 270000 query time 106.03s 
🍒 Sys[out] » Total votes 300000 query time 117.93s 
🍒 Sys[out] » Total votes 330000 query time 129.41s 
🍒 Sys[out] » Total votes 360000 query time 141.59s 
🍒 Sys[out] » Total votes 390000 query time 152.61s 
🍒 Sys[out] » Total votes 420000 query time 163.8s 
🍒 Sys[out] » Total votes 450000 query time 175.97s 
🍒 Sys[out] » Total votes 480000 query time 186.95s 
🍒 Sys[out] » Total votes 481925 query time 187.93s 
🍒 Cereja » [▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰] - 100.00% - 🕜 00:03:08 total - Done! ✅ 


In [10]:
# Filtrar votos desde a ultima clusterizaçãão 
# BZS -> (1 de julho)
# ASL ->
delta_time =  timedelta(days=27)
today = datetime.now() - delta_time

votes_true = {}

for key,value in votes.items():
    if (value['score']>1) and (value['automaticMatch']==False) and (value['lastUpdate'].timestamp() > today.timestamp()):
        votes_true[key] = value
votes = votes_true
print(today)

2021-06-04 20:28:47.846927


# Funções de operação nos clusters

### AMBOS OS SEGMENTOS SEM CLUSTER

In [17]:
# Caso nenhum dos segmentos tenha cluster, um novo sinal é criado
def both_without_cluster(segment_a,segment_b):
    set_operations = []
    update_operations = []

    # segmentos
    segment_a_doc = cmt.get_document('segments',segment_a)
    segment_b_doc = cmt.get_document('segments',segment_b)

    #videos sentences
    video_a_sentence = segment_a_doc.video.sentence
    video_b_sentence = segment_a_doc.video.sentence

    #TF_idf_cluster
    cluster_sentence = ' '.join([video_a_sentence,video_b_sentence])
    cluster_words = tf_idf.sentence_tf_idf(cluster_sentence, use_filter=True,stop_words=stop)
    cluster_words = ordered_words(cluster_words)[:5]
    words = [{word_score[0]: word_score[1]} for word_score in cluster_words]
    relatedWords = []
    for word_score in cluster_words:
        relatedWords.append(word_score[0])

    #criando novo sinal
    sign_col = cmt.firestore.db.collection('workspaces').document(WORKSPACE).collection('signs')
    id_doc = cmt.firestore.db.collection('whatever').document().id
    sign_ref = sign_col.document(id_doc)
    sign_data = deepcopy(sign_template)
    sign_data['primarySegment'] = segment_a_doc.reference
    sign_data['numberOfSegments'] = 2
    sign_data['words'] = words
    sign_data['relatedWords'] = relatedWords
    set_operations.append((sign_ref,sign_data))
    # print(set_operations[0][0].path,set_operations)
    cmt.firestore.set(set_operations,safe_mode=False)

    #Atualizando segmentos
    update_operations.append((segment_a_doc.reference, {'movement': sign_ref,'hasMovement':True}))
    update_operations.append((segment_b_doc.reference, {'movement': sign_ref,'hasMovement':True}))
    # print(update_operations[0][0].path,update_operations)
    cmt.firestore.update(update_operations,safe_mode=False)

### Apenas 1 dos segmentos tem cluster

In [18]:
# caso um dos dois tenha cluster, o sinal que nãão tem éé adicionado a esse sinal
def one_has_cluster(segment_a,segment_b):
    update_operations = []
    #segmentos
    segment_a_doc = cmt.get_document('segments',segment_a)
    segment_b_doc = cmt.get_document('segments',segment_b)

    #sinal
    sign_ref = segment_a_doc.movement

    if sign_ref._is_definitive:
        # devemos ter cuidados com atualização desses sinais
        update_operations.append((sign_ref.reference, {'numberOfSegments': sign_ref.number_of_segments + 1}))
    else:
        #reconstruir cluster
        where = [('movement', '==', sign_ref.reference)]
        segments = cmt.list_collection('segments', where_config=where)

        #resgatando sentenças
        sentences = [segment.video.sentence for _,segment in segments.items()]
        sentences.append(segment_b_doc.video.sentence)

        #TF_idf_cluster
        cluster_sentence = ' '.join(sentences)
        cluster_words = tf_idf.sentence_tf_idf(cluster_sentence, use_filter=True,stop_words=stop)
        cluster_words = ordered_words(cluster_words)[:5]
        words = [{word_score[0]: word_score[1]} for word_score in cluster_words]
        relatedWords = []
        for word_score in cluster_words:
            relatedWords.append(word_score[0])

        #atualizando sinal
        update_operations.append((sign_ref.reference, {'numberOfSegments': sign_ref.number_of_segments + 1,'words': words, 'relatedWords': relatedWords}))
    
    #atualizando segmento novo
    update_operations.append((segment_b_doc.reference, {'movement': sign_ref.reference,'hasMovement':True}))
    
    #subindo atualizaçãão
    cmt.firestore.update(update_operations,safe_mode=False)

### Ambos os segmentos tem cluster

In [19]:
def update_both_movements(segment_a_doc,movement_a_doc,segment_b_doc,movement_b_doc):
    update_operations = []
    # reconstruir cluster
    where = [('movement', '==', movement_b_doc.reference)]
    segments_b = cmt.list_collection('segments', where_config=where)
    where = [('movement', '==', movement_a_doc.reference)]
    segments_a = cmt.list_collection('segments', where_config=where)

    if movement_a_doc._is_definitive:
        # devemos ter cuidados com atualização desses sinais
        update_operations.append((movement_a_doc.reference, {'numberOfSegments': movement_a_doc.number_of_segments + movement_b_doc.number_of_segments}))
    else:
        #TFIDF
        #resgatando sentenças
        sentences = [segment.video.sentence for _,segment in segments_a.items()] + [segment.video.sentence for _,segment in segments_b.items()]

        #TF_idf_cluster
        cluster_sentence = ' '.join(sentences)
        cluster_words = tf_idf.sentence_tf_idf(cluster_sentence, use_filter=True, stop_words=stop)
        cluster_words = ordered_words(cluster_words)[:5]
        words = [{word_score[0]: word_score[1]} for word_score in cluster_words]
        relatedWords = []
        for word_score in cluster_words:
            relatedWords.append(word_score[0])

        update_operations.append((movement_a_doc.reference, {'words': words, 'relatedWords': relatedWords,'numberOfSegments': movement_a_doc.number_of_segments + movement_b_doc.number_of_segments }))

    #UPDATE no cluster "vencedor"
    for segment_id,segment in segments_b.items():
        update_operations.append((segment.reference, {'movement': movement_a_doc.reference,'hasMovement':True}))
    
    update_operations.append((movement_b_doc.reference, {'hasCluster': False,'numberOfSegments': 0 }))
    cmt.firestore.update(update_operations,safe_mode=False)

In [20]:
def link_definitive_movements(movement_a_doc,movement_b_doc):
    update_operations = [(movement_b_doc.reference, {'signDefinitiveRelation':movement_a_doc.reference})]
    cmt.firestore.update(update_operations,safe_mode=False)

In [21]:
def both_has_cluster(segment_a,segment_b):
    #segments
    segment_a_doc = cmt.get_document('segments',segment_a)
    segment_b_doc = cmt.get_document('segments',segment_b)

    #movements
    movement_a_doc = segment_a_doc.movement
    movement_b_doc = segment_b_doc.movement


    # Se ambos são definitvos
    ### Esse ponto é critico, deve SER CRIADA A CONEXAO ENTRE MOVIMENTOS DEFINITIVOS
    if movement_a_doc._is_definitive and movement_b_doc._is_definitive:
        print(movement_a_doc.reference.path,'->>',movement_b_doc.reference.path)
        # teste = input('sãão caras definitivos, cuidado')
        # se nenhum dos dois movimentos tem erro, escolhe-se o que tem mais segmentos
        if (not movement_a_doc.current_animation_has_errors) and (not movement_b_doc.current_animation_has_errors):
            if movement_a_doc.number_of_segments > movement_b_doc.number_of_segments:
                link_definitive_movements(movement_a_doc,movement_b_doc)
            else:
                link_definitive_movements(movement_b_doc,movement_a_doc)

        # se um dos dois movimentos tem erro, escolhe-se o outro
        elif not movement_a_doc.current_animation_has_errors:
            link_definitive_movements(movement_a_doc,movement_b_doc)
        elif not movement_b_doc.current_animation_has_errors:
            link_definitive_movements(movement_b_doc,movement_a_doc)
        
        # Se ambos tem erros
        else:
            if movement_a_doc.number_of_segments > movement_b_doc.number_of_segments:
                link_definitive_movements(movement_a_doc,movement_b_doc)
            else:
                link_definitive_movements(movement_b_doc,movement_a_doc)


    # Se apenas o segmento A éé definitivo
    elif movement_a_doc._is_definitive:
        print(movement_a_doc.reference.path)
        # teste = input('éé a definitivo, cuidado')
        update_both_movements(segment_a_doc,movement_a_doc,segment_b_doc,movement_b_doc)

    # Se apenas o Segmento B éé definitivo
    elif movement_b_doc._is_definitive:
        print(movement_b_doc.reference.path)
        # teste = input('éé b definitivo, cuidado')
        update_both_movements(segment_b_doc,movement_b_doc,segment_a_doc,movement_a_doc)

    # Caso ambos nãão sejam definitivos, mas sejam promovidos
    elif movement_a_doc.promoted and movement_b_doc.promoted:
        if movement_a_doc.number_of_segments > movement_b_doc.number_of_segments:
            update_both_movements(segment_a_doc,movement_a_doc,segment_b_doc,movement_b_doc)
        else:
            update_both_movements(segment_b_doc,movement_b_doc,segment_a_doc,movement_a_doc)

    # Caso somente o segmento A seja promovido
    elif movement_a_doc.promoted:
        update_both_movements(segment_a_doc,movement_a_doc,segment_b_doc,movement_b_doc)

    # Caso somente o segmento B seja promovido
    elif movement_b_doc.promoted:
        update_both_movements(segment_b_doc,movement_b_doc,segment_a_doc,movement_a_doc)

    # Caso nenhum seja promovido
    else:
        if movement_a_doc.number_of_segments > movement_b_doc.number_of_segments:
            update_both_movements(segment_a_doc,movement_a_doc,segment_b_doc,movement_b_doc)
        else:
            update_both_movements(segment_b_doc,movement_b_doc,segment_a_doc,movement_a_doc)

# Atualizar Clusters

In [ ]:
# def create_vote_doc(seg_a: str, seg_b: str):
#     vote_doc = {
#         'segment_a': seg_a,'segment_b': seg_b,
#         'score': 1, 'votes': 1, 
#         'createdAt': datetime.now(), 'lastUpdate': datetime.now(),
#         'automaticMatch': False
#     }
#     return vote_doc

In [ ]:
# Entrada
# isso vai vir do community
# seg_left = None 
# segs_right = 10*[None] # teremos uma lista [seg_2a, seg_2b, ..., seg_2j]

# # 'votes' terá que ser o resultado de uma query na coleção de votos
# for seg_r in segs_right:
#     segs = sorted([seg_left, seg_r])
#     seg_a = segs[0]
#     seg_b = segs[1]

#     query = [('segment_a', '==', seg_a),
#              ('segment_b', '==', seg_b)]

#     vote = cmt.list_longer_collection('votes', where_config=query)

#     if not vote:
#         # voto não existe - criar
#         vote = create_vote_doc(seg_a, seg_b)

#     assert len(vote) == 1, f'Erro! Mais de um voto encontrado com {seg_a} e {seg_b}'

    # Segue normal com o código do Lucas

In [22]:
from tqdm import tqdm
from IPython.display import clear_output

# Main

# votes = cmt.list_longer_collection('votes')

for _,vote in tqdm(votes.items()):
    # score insuficiente
    if vote['score'] < 2:
        clear_output(wait = True)
        continue
    else:
        #segmentos
        segment_a = cmt.get_document('segments',vote['segment_a'])
        segment_b = cmt.get_document('segments',vote['segment_b'])

        # ambos os segmentos ja tem o mesmo sinal
        if (segment_a.has_movement) and (segment_b.has_movement) and (segment_a.movement.reference == segment_b.movement.reference):
            clear_output(wait = True)
            continue
        #nenhum dos dois tem sinal
        elif not (segment_a.has_movement) and not (segment_b.has_movement):
            both_without_cluster(vote['segment_a'],vote['segment_b'])
        #um dos dois tem sinal
        elif (segment_a.has_movement) and not (segment_b.has_movement):
            one_has_cluster(vote['segment_a'],vote['segment_b'])
        elif not (segment_a.has_movement) and (segment_b.has_movement):
            one_has_cluster(vote['segment_b'],vote['segment_a'])
        #ambos tem sinal
        else:
            both_has_cluster(vote['segment_a'],vote['segment_b'])
        clear_output(wait = True)

100%|██████████| 1951/1951 [27:20<00:00,  1.19it/s]


# Update votes
Precisamos atualizar os respectivos clusters na coleção de votos

## Resgatando votos e segmentos

In [25]:
votes = cmt.list_longer_collection('votes')
all_segments_docs = cmt.list_longer_collection('segments')
clear_output()
segments = {s: d for s, d in all_segments_docs.items() if d['hasMovement'] == True}

In [26]:
update_operations = []
for vote_id,vote in votes.items():
    try:
        if vote['score'] > 1:
            if segments[vote['segment_a']]['movement'].id == segments[vote['segment_b']]['movement'].id:
                update_operations.append((vote['ref'],{'currentMovement':segments[vote['segment_a']]['movement'].id}))
            else:
                update_operations.append((vote['ref'],{'currentMovement':''}))
        else:
            update_operations.append((vote['ref'],{'currentMovement':''}))
    except:
        update_operations.append((vote['ref'],{'currentMovement':''}))

In [27]:
cmt.firestore.update(update_operations)

Digite sim para confirmar a alteração em 481925 registros: sim
🍒 Cereja » [▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰▰] - 100.00% - 🕜 00:06:03 total - Done! ✅ 
🍒 Cereja » Awaiting... 

# TF-IDF update

In [29]:
query = [('errors', '==', None)]
videos_docs = cmt.list_longer_collection('videos', where_config=query)
clear_output()
len(videos_docs)

113093

In [30]:
# filtrando apenas vídeos que possuam algum segmento em cluster
videos = {}
for s, s_data in all_segments_docs.items():
    v = s_data['video'].id
    if s_data['hasMovement'] and v not in videos and v in videos_docs:
        videos[v] = videos_docs[v]
len(videos)

20299

In [31]:
from cereja import Progress, TfIdf
import pickle

sentences = [video['sentence'] for _,video in videos.items()]
# ADD 'tom, toms, mary, marys' em stopwords
stop = ['em', 'do', 'da', 'de', 'pro', 'pra', 'no', 'dos', 'na', 'os', 'à', 'pela', 'nas', 'num',\
        'das', 'numa', 'nos', 'às', 'pelas', 'as', 'uns', 'umas', 'lhe', 'é', 'o', 'a', 'tom', 'mary', 'toms', 'marys',
        'is', 'are', 'am', 'at', 'an', 'of', 'the'] if WORKSPACE == 'HT-BZS' \
        else \
        ['em', 'da', 'de', 'pro', 'pra', 'no', 'dos', 'na', 'os', 'à', 'pela', 'nas', 'num',\
        'das', 'numa', 'nos', 'às', 'pelas', 'as', 'uns', 'umas', 'lhe', 'é', 'o', 'a', 'tom', 'mary', 'toms', 'marys',
        'is', 'are', 'am', 'at', 'an', 'of', 'the']

tf_idf = TfIdf(sentences, stop_words=stop)

# Save TF_IDF into a pickle file.

pickle.dump( tf_idf, open( f"TF_IFD_{WORKSPACE}.pkl", "wb" ) )

blob = cmt.firebase.storage.db.blob(f'modelos/automatic_cluster/TF_IFD_{WORKSPACE}.pkl')
blob.upload_from_filename(f"TF_IFD_{WORKSPACE}.pkl")

10681 words on corpus
Computing idf...
idf computed!
